In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x = torch.randn(B, T, C)
x.shape

Токен на 5 позиции должен обзщаться с 1-4.
Простой способ - взять среднее значение предыдущих элементов.
Теряется много инофрмации о расположении.
Мы просто взвешиваем информацию о текущем токене и о всех предыдущих.
Исопльзуем нижню треугольную матрицу, чтобы максировать 
bag of words -- 

In [ ]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [ ]:
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(c)

In [ ]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print(c)

# version 3

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))

# какие-то токены будут более интересны для текущего токена, взвешенная агрегация токенов
wei = torch.zeros((T,T)) # насколько каждый токен в прошлом комуницирует с текущим
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

## Self-attention one head

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False) # элементы которые мы агрегируем
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

In [ ]:
k.shape

In [ ]:
q.shape

In [ ]:
wei[0]